In [8]:
import json
import pandas as pd

#load the jason file
file_path = r"C:\Users\sajad\Downloads\2armed_bandit.json"
with open(file_path, "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.head()

,rt,stimulus,response,trial_type,trial_index,time_elapsed,internal_node_id,bean_trial_duration,bean_duration,bean_stimulus,...,bean_text,bean_color,bean_correct,value,bean_html_array,bean_values,bean_time_after_response,bean_bandits,bean_value,bean_score
0,6946.0,<div style='color:white'>Welcome to our decisi...,,html-keyboard-response,0,6951,0.0-0.0-0.0,NaN,NaN,<div style='color:white'>Welcome to our decisi...,...,Welcome to our decision-making experiment. In ...,white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1287.0,"<div style='color:white'>\n <div class=""slo...",,html-keyboard-response,1,8241,0.0-0.0-1.0,NaN,NaN,"<div style='color:white'>""\n <div class=""sl...",...,"""\n <div class=""slotmachine"" style=""positio...",white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1130.0,<div style='color:white'>After selecting a ban...,,html-keyboard-response,2,9372,0.0-0.0-2.0,NaN,NaN,<div style='color:white'>After selecting a ban...,...,"After selecting a bandit, you will receive fee...",white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,520.0,<div style='color:white'>Your goal is to earn ...,,html-keyboard-response,3,9894,0.0-0.0-3.0,NaN,NaN,<div style='color:white'>Your goal is to earn ...,...,Your goal is to earn as many points as possibl...,white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1414.0,"[<div class=""slotmachine"" style=""position: abs...",0,html-choice,4,13317,0.0-1.0-0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,"[<div class=""slotmachine"" style=""position: abs...","[0, 1]",2000.0,"[{'color': 'orange', 'value': 0}, {'color': 'b...",0.0,0.0
